In [6]:
!pip install earthengine-api crewai psycopg2-binary pandas numpy scipy

  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl.metadata (1.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.1 MB/s eta 0:00:00
  Using cached alembic-1.13.2-py3-none-any.whl.metadata (7.4 kB)
  Using cached posthog-3.6.5-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached pyparsing-3.1.4-py3-none-any.whl.metadata (5.1 kB)
  Using cached langsmith-0.1.120-py3-none-any.whl.metadata (13 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.27.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached opentelemetry_semantic_conventions-0.48b0-py3-none-any.whl.metadata (2.4 kB)
  Using cached fastapi-0.114.2-py3-none-any.whl.metadata (27 kB)
  Using cached onnxruntime-1.19.2-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.27.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_instrumentat

In [7]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Set OpenAI API Key as environment variable
import os
os.environ['OPENAI_API_KEY'] = 'Provide your key here'  # Replace with your actual OpenAI API key

In [13]:
# Import necessary libraries
import ee
import numpy as np
import psycopg2
from psycopg2 import sql
import crewai
from crewai.agent import Agent
from pydantic import Field, ValidationError

# CrewAI Agent for exporting Sentinel-2 bands to PostgreSQL
class Sentinel2ExportAgent(Agent):
    project_id: str = Field(..., description="Project ID for Earth Engine")
    db_url: str = Field(..., description="PostgreSQL URL for Database")

    def __init__(self, name, role, goal, backstory, project_id, db_url):
        try:
            super().__init__(name=name, role=role, goal=goal, backstory=backstory, project_id=project_id, db_url=db_url)
        except ValidationError as e:
            print(e)
            return

        self.project_id = project_id
        self.db_url = db_url
        self.authenticate_and_initialize_earth_engine()

    def authenticate_and_initialize_earth_engine(self):
        try:
            # Authenticate the user
            ee.Authenticate()  # This will prompt OAuth authentication

            # Initialize Earth Engine with the specified project ID
            ee.Initialize(project=self.project_id)
            print(f"Earth Engine initialized with project: {self.project_id}")
        except Exception as e:
            print(f"Error during authentication or initialization: {e}")

    def store_bands_in_db(self, latitude, longitude, start_date, end_date, bands_table_name):
        try:
            # Define a point around which to get the image
            point = ee.Geometry.Point([longitude, latitude])

            # Load the Sentinel-2 image collection and filter by date and location
            collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                          .filterBounds(point)
                          .filterDate(start_date, end_date)
                          .sort('CLOUD_COVER', True))

            # Get the first image in the filtered collection
            image = collection.first()

            # Extract all 13 bands
            bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']
            band_values = image.select(bands)

            # Define the region (1 km buffer around the point)
            region = point.buffer(1000).bounds()

            # Get band data as list
            band_data = band_values.reduceRegion(
                reducer=ee.Reducer.toList(),
                geometry=region,
                scale=30
            ).getInfo()

            # Convert to numpy arrays
            band_data_np = {band: np.array(band_data[band]) for band in bands}

            # Remove existing data in the PostgreSQL table and insert new data
            self.remove_existing_data(bands_table_name)
            self.insert_bands_into_postgres(bands_table_name, band_data_np)

        except Exception as e:
            print(f"Error during band data storage: {e}")

    def remove_existing_data(self, table_name):
        try:
            # Connect to PostgreSQL database
            conn = psycopg2.connect(self.db_url)
            cursor = conn.cursor()

            # Truncate the table (delete all rows) to remove existing data
            cursor.execute(sql.SQL(f"TRUNCATE TABLE {table_name};"))

            # Commit and close the connection
            conn.commit()
            cursor.close()
            conn.close()

            print(f"Existing data in table '{table_name}' removed.")

        except Exception as e:
            print(f"Error removing existing data from PostgreSQL: {e}")

    def insert_bands_into_postgres(self, table_name, band_data):
        try:
            # Connect to PostgreSQL database
            conn = psycopg2.connect(self.db_url)
            cursor = conn.cursor()

            # Create table for 13 Sentinel-2 bands if it doesn't exist
            cursor.execute(sql.SQL(f"""
                CREATE TABLE IF NOT EXISTS {table_name} (
                    id SERIAL PRIMARY KEY,
                    B1 FLOAT[],
                    B2 FLOAT[],
                    B3 FLOAT[],
                    B4 FLOAT[],
                    B5 FLOAT[],
                    B6 FLOAT[],
                    B7 FLOAT[],
                    B8 FLOAT[],
                    B8A FLOAT[],
                    B9 FLOAT[],
                    B10 FLOAT[],
                    B11 FLOAT[],
                    B12 FLOAT[]
                );
            """))

            # Insert band data into table
            cursor.execute(sql.SQL(f"""
                INSERT INTO {table_name} (B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
            """), [band_data[band].tolist() for band in band_data])

            # Commit and close the connection
            conn.commit()
            cursor.close()
            conn.close()

            print(f"Band data inserted into table '{table_name}'.")

        except Exception as e:
            print(f"Error inserting band data into PostgreSQL: {e}")

In [14]:
# Main code execution
if __name__ == "__main__":

    # Set up project details and database URL
    project_id = 'Provide your project ID here'  # Replace with your actual project ID
    db_url = 'Provide your database url here'  # Replace with your actual database URL

    # Define the role, goal, and backstory for the agent
    role = "Data Analyst"
    goal = "Store Sentinel-2 bands in PostgreSQL database"
    backstory = "The agent assists in data analysis by exporting Sentinel-2 bands for further analysis."

    # Create and run the CrewAI agent
    sentinel2_agent = Sentinel2ExportAgent(
        name="Sentinel2ExportAgent",
        role=role,
        goal=goal,
        backstory=backstory,
        project_id=project_id,
        db_url=db_url
    )

    # Example usage: store Sentinel-2 bands in PostgreSQL
    latitude = 37.7749   # Latitude for San Francisco
    longitude = -122.4194  # Longitude for San Francisco
    start_date = '2021-06-01'
    end_date = '2021-06-30'
    bands_table_name = 'sentinel2_bands_data'

    sentinel2_agent.store_bands_in_db(latitude, longitude, start_date, end_date, bands_table_name)


Earth Engine initialized with project: genai-agent-hack-2024
Existing data in table 'sentinel2_bands_data' removed.
Band data inserted into table 'sentinel2_bands_data'.
